# WF

In [34]:
%reload_ext autoreload
%autoreload 2
import colabexts
from colabexts.jcommon import *
import xmltodict
import colabexts.utils as colabexts_utils


import os, sys, datetime, re, json, importlib
from collections import defaultdict
from sys import modules
from IPython.display import HTML, Javascript
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

%matplotlib inline
pd.set_option('display.max_colwidth', 40)
pd.set_option('display.max_rows', 16)

import logging
for handler in logging.root.handlers[:]: logging.root.removeHandler(handler)
logging.basicConfig( level=logging.INFO,
        format='%(asctime)s %(name)s %(levelname)s: %(message)s',
        #handlers=[ logging.FileHandler("/tmp/stream.log"), logging.StreamHandler()],
        handlers=[ logging.StreamHandler()],
)
logger = logging.getLogger( "geoapp" )
logger.info( f"Testing logging" )

2024-07-12 15:44:23,672 geoapp INFO: Testing logging


# WF_services

In [51]:
%%writefile ../wf_services.py
#!/usr/bin/env python 

#*** DO NOT EDIT - GENERATED FROM tseries/notebooks/tseries_services.ipynb ****

import sys, os,  glob, logging, datetime, json
from  mangorest.mango import webapi
from colabexts import utils as colabexts_utils
import pandas as pd
import xmltodict
import colabexts.utils as colabexts_utils

sys.path.append("../..")

MBASE  = "/opt/data/data/workflows/"
logger = logging.getLogger( "geoapp" )

#---------------------------------------------------------------------------------    
@webapi("/wf/save_wf")
def save_wf( name, jsn, xml, **kwargs):
    name = name or "default"
    if (name.endswith(".json")):  name = name[:-5]
    if (name.endswith(".xml")):   name = name[:-4]
        
    if not os.path.exists(MBASE ):
        os.makedirs(MBASE)
    
    with open(f"{MBASE}/{name}.xml",  "w") as f:   f.write(xml.strip())
    if ( len(jsn) < 30 ):
        jsn = xmltodict.parse(xml,attr_prefix='', cdata_key='#text')
        jsn = json.dumps(jsn, indent=4)

    with open(f"{MBASE}/{name}.json", "w") as f:   f.write(jsn.strip())

    return f"Saved "    
#---------------------------------------------------------------------------------    
@webapi("/wf/get_wf")
def get_wf( name, **kwargs):
    if (name.endswith(".json")):  name = name[:-5]
    if (name.endswith(".xml")):   name = name[:-4]

    xml = open(f"{MBASE}/{name}.xml",  "r").read()
    ret = dict(json="", xml=xml)

    return ret  
#------------------------------------------------------------------------------
@webapi("/wf/delete_wf")
def delete_wf(name="", **kwargs):
    if (name.endswith(".json")):  name = name[:-5]
    if (name.endswith(".xml")):   name = name[:-4]

    logger.info(f"DELETE {name}")
    
    dst = f"{MBASE}/{name}.xml"
    if ( os.path.exists(dst) ):  os.remove(dst)

    dst = f"{MBASE}/{name}.json"
    if ( os.path.exists(dst) ):  os.remove(dst)

    return f"Deleted {dst}."
#------------------------------------------------------------------------------
@webapi("/wf/getall_wfs")
def getAllWorkflows( nrows=10000, patt='*.xml', **kwargs):
    
    files = glob.glob(f"{MBASE}{patt}")
    files.sort(key=os.path.getmtime, reverse=True)

    ret = {
        "name": "workflows",
        "columns": ["name"],
        "values": [[f[len(MBASE):]] for f in files][0:nrows]
    }
    return ret
#---------------------------------------------------------------------------------    
@webapi("/wf/run_wf")
def run_wf( name, **kwargs):
    if (name.endswith(".json")):  name = name[:-5]
    if (name.endswith(".xml")):   name = name[:-4]

    wfdef  = f"/opt/data/data/workflows/{name}.xml"
    buf=open(wfdef,"r").read()
    wf = xmltodict.parse(buf,attr_prefix='', cdata_key='#text')


    wfsh="#Execute\n"
    for c in wf['mxGraphModel']['root']['mxCell']:
        w = colabexts_utils.Map(c)
        s = w.text or ""
        wfsh += f"#Processing ID: {w.id}, Value: {w.value} => Cmd: {s}\n"
        if (not s):
            continue
        wfsh += f"{s}\n"

    open("/tmp/run.sh","w").write(wfsh)

    return f"Running /tmp/run.sh {wfsh} "

Overwriting ../wf_services.py


In [108]:
name='sada'
wfdef  = f"/opt/data/data/workflows/{name}.xml"
buf=open(wfdef,"r").read()
wf = xmltodict.parse(buf,attr_prefix='', cdata_key='#text')


#==================================================================================
def startRunning(w):
    if (w.state == 'running'):
        print("CYCLE DETECTED")
        return ""
    w.state = 'runnning'
    ret=f"#=> ID: {w.id}, Value: {w.value} => Config: {w.config}\n{w.config.get('cmd','')}"
    print(ret)
    for e in w.edges or []:
        if ( e.target != w):
            ret += startRunning(e.target)
    return ret

#==================================================================================
nodes = {}
for c in wf['mxGraphModel']['root']['mxCell']:
    w = colabexts_utils.Map(c)
    nodes[w.id] = w
    w.state = 'init'
    cfg = w.config or "{}"
    try:
        cfg = re.subn('//.*\n', '\n', cfg)[0]
        cfg=colabexts_utils.parsej(cfg)
        w.config = cfg
    except:
        w.config = {}
        pass
    if (not w.edge):
        w['edges']  = []


startNodes = []
for k,w in nodes.items():
    if (w.edge):
       w['source']  = nodes[w['source']]
       w['target']  = nodes[w['target']]
       w['source']['edges'].append(w)
       w['target']['edges'].append(w)
    else:
        typ = w['config'].get("type", "")
        if ( typ == 'start'):
            startNodes.append(w)

wfsh="#Execute:\n"
if (len(startNodes) <=0):
    print("NO START NODES")
else:
    i = 0
    for w in startNodes:
        startRunning(w)

print(wfsh)


#=> ID: 13, Value: S => Config: {'type': 'start', 'run': '10-20', 'nrun': 1, 'cmd': 'echo `date` > a.txt'}
echo `date` > a.txt
#=> ID: 2, Value: Start => Config: {'type': 'na|process|start', 'run': '[10,20]', 'nrun': 1, 'cmd': 'echo -- `date` >> a.txt'}
echo -- `date` >> a.txt
#=> ID: 11, Value: System Admin => Config: {'type': 'na|process|start', 'run': '[10,20]', 'nrun': 1}

#=> ID: 5, Value: Send Email! OK => Config: {}

#=> ID: 4, Value: Build Model! => Config: {'type': 'na|process|start', 'run': '[10,20]', 'nrun': 1, 'cmd': 'echo BUILD MODEL >> a.txt'}
echo BUILD MODEL >> a.txt
#=> ID: 3, Value: Process Data! => Config: {}

#=> ID: 10, Value:  => Config: {'type': 'na|process|start', 'run': '[10,20]', 'nrun': 1, 'cmd': 'echo LAST EMA MODEL >> a.txt'}
echo LAST EMA MODEL >> a.txt
#Execute:



In [101]:
for k,w in nodes.items():
    print(w.id, w.config, w.edges)

0 {} []
1 {} []
2 {} [{'id': '12', 'value': '', 'style': 'strokeWidth=1;strokeColor=#808080;', 'parent': '1', 'source': {'id': '2', 'value': 'Start', 'style': 'fillColor=#d4e1f5;', 'parent': '1', 'vertex': '1', 'state': 'init', 'mxGeometry': {'x': '130', 'y': '100', 'width': '120', 'height': '40', 'as': 'geometry'}, 'config': {}, 'edges': [...]}, 'target': {'id': '11', 'value': 'System Admin', 'style': 'shape=actor;fillColor=#d4e1f5;', 'parent': '1', 'vertex': '1', 'text': 'Validate and Verify\xa0Is Approved?SADA', 'state': 'init', 'mxGeometry': {'x': '410', 'y': '140', 'width': '30', 'height': '40', 'as': 'geometry'}, 'config': {}, 'edges': [{'id': '8', 'value': '', 'style': 'strokeWidth=1;strokeColor=#808080;', 'parent': '1', 'source': {...}, 'target': {'id': '5', 'value': 'Send Email! OK', 'style': 'fillColor=#d4e1f5;', 'parent': '1', 'vertex': '1', 'text': 'SADANNADA', 'state': 'init', 'mxGeometry': {'x': '730', 'y': '310', 'width': '120', 'height': '40', 'as': 'geometry'}, 'config

{'type': 'na|process|start', 'run': '10-20', 'nrun': 1}

("{\n  type: 'na|process|start', \n start\n  run :  '10-20',  \n run time(secs)\n  nrun: 1         \n can run ntimes\n}",
 0)